In [ ]:
%%capture
!pip install datasets transformers accelerate
!pip install optuna
!apt install git-lfs
!pip install -q kaggle

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title Global Params
base_model = "AIMH/mental-bert-base-cased" #@param {type:"string"}
path_to_data = "/content/drive/MyDrive/Data/merged_data.csv" # @param ["/content/drive/MyDrive/Data/Filtered-Mental-Health-Twitter.csv", "/content/drive/MyDrive/Data/data.csv"] {allow-input: true}
repo_name = "BERTForDetectingDepression-Twitter2020" # @param ["BERTForDetectingDepression-Twitter2015", "BERTForDetectingDepression-Twitter2020"] {allow-input: true}

In [ ]:
from google.colab import drive
from datasets import load_dataset, load_metric

drive.mount("/content/drive/")

full_dataset = load_dataset("csv", data_files = path_to_data, split="train", trust_remote_code=True)
dataset = full_dataset.train_test_split(test_size=0.1)
print(dataset["train"][0])

Mounted at /content/drive/


Generating train split: 0 examples [00:00, ? examples/s]

{'user_id': 1306322067395096584, 'post_text': 'im one more heartbreak away from posting nba youngboy quotes on my story. https://t.co/YWd9ey899e. idk why i keep expecting not to get my feelings hurt🧏🏾\u200d♀️. my asl teacher is buggin....i hate this bitch. go follow ma managa😆 @DaddyIanHawke. i want chipotle. NAE NAE😁🙏🏾 https://t.co/VamisqFnLU. down ATROCIOUS. https://t.co/ElscVi6Foj. there are so many things i just cant wrap my head around rn like 🤔🤔. arent vegans just glorified herbivores?. i don’t even know why i try anymore. THEODORE WHAT THE FUCK BRO. im so sad idk why i cause situations knowing that im gonna get my feelings hurt😕🤟🏾‼️. @dfw_ken no☺️. THATS NOT ME https://t.co/9BV47JwSxP. @dfw_ken sigh😟. someone tell @hbk_chelsea dat im not interested😐. @Vanessa23931706 PLEASEEHKDDJDK WE WILL BE FRIENDS W YOU. that was the end of the tweet. nothing else comes after https://t.co/9aS5slV2dP. i’m at my limit🙅🏾\u200d♀️. i would express my emotions but i think im going to refrain for aw

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['user_id', 'post_text', 'label'],
        num_rows: 1010
    })
    test: Dataset({
        features: ['user_id', 'post_text', 'label'],
        num_rows: 113
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def preprocess_fn(examples):
    return tokenizer(examples["post_text"], truncation = True)

encoded_dataset = dataset.map(preprocess_fn, batched = True)
encoded_dataset

Map:   0%|          | 0/1010 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['user_id', 'post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1010
    })
    test: Dataset({
        features: ['user_id', 'post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 113
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

args = TrainingArguments(
    repo_name,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 6.780781971197275e-05,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 16,
    num_train_epochs = 4,
    weight_decay = 0.05,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    push_to_hub = False
)

In [ ]:
import numpy as np

metric = load_metric("accuracy", trust_remote_code = True)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis = 1)
    return metric.compute(predictions = preds, references = labels)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(base_model, num_labels = 2)


trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset = encoded_dataset["train"].shard(index = 1, num_shards = 10),
    eval_dataset = encoded_dataset["test"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_run = trainer.hyperparameter_search(n_trials = 10, direction = "maximize")

[I 2024-08-02 05:22:05,446] A new study created in memory with name: no-name-d4719e14-b4bf-42d3-8c6c-f60135387ada
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.760988,0.398230
2,No log,0.710453,0.407080


[I 2024-08-02 05:22:23,356] Trial 0 finished with value: 0.40707964601769914 and parameters: {'learning_rate': 8.449706849507742e-05, 'num_train_epochs': 2, 'seed': 14, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.40707964601769914.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.879027,0.398230
2,No log,0.808834,0.398230
3,No log,0.758562,0.407080


[I 2024-08-02 05:22:45,481] Trial 1 finished with value: 0.40707964601769914 and parameters: {'learning_rate': 1.6221451008103506e-05, 'num_train_epochs': 3, 'seed': 29, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.40707964601769914.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.838182,0.398230
2,No log,0.745499,0.398230


[I 2024-08-02 05:24:55,349] Trial 8 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.805012,0.407080


[W 2024-08-02 05:24:59,155] Trial 9 failed with parameters: {'learning_rate': 1.9185267410476043e-05, 'num_train_epochs': 3, 'seed': 9, 'per_device_train_batch_size': 64} because of the following error: TypeError("unsupported operand type(s) for /: 'int' and 'NoneType'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 211, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1932, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2365, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-

TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'

In [ ]:
best_run

NameError: name 'best_run' is not defined

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
trainer.train_dataset = encoded_dataset["train"]
# setattr(trainer.args, "num_train_epochs", 2)

In [ ]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIMH/mental-bert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.665087,0.610619
2,No log,0.910256,0.407080
3,No log,0.927886,0.522124
4,No log,0.928410,0.628319


TrainOutput(global_step=104, training_loss=0.5761726819551908, metrics={'train_runtime': 27.4002, 'train_samples_per_second': 14.744, 'train_steps_per_second': 3.796, 'total_flos': 106296866365440.0, 'train_loss': 0.5761726819551908, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6675335764884949,
 'eval_accuracy': 0.6415,
 'eval_runtime': 2.7347,
 'eval_samples_per_second': 731.33,
 'eval_steps_per_second': 45.708,
 'epoch': 4.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1722570954.8953887c6028.7787.11:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Silicon23/BERTForDetectingDepression-Twitter2020-baseBERT/commit/309cad4dae5e2ca74ff41fa7f6df00d07cf047ed', commit_message='End of training', commit_description='', oid='309cad4dae5e2ca74ff41fa7f6df00d07cf047ed', pr_url=None, pr_revision=None, pr_num=None)